In [ ]:
import requests
import json
import random
import time

# Base URL of the FastAPI server
BASE_URL = "http://localhost:8000"

# Optional: user_id for personalized /locations/recommended
# Get this from /auth/login if you want to test recommendations
USER_ID = None  # e.g. "6e0e5c7e-1234-4567-890a-bcdef1234567"

HEADERS = {"X-User-ID": USER_ID} if USER_ID else {}

print("BASE_URL =", BASE_URL)
print("USER_ID =", USER_ID)


In [ ]:
location_names = [
    "Victoria Peak", "Lion Rock", "Dragon's Back", "Mount Parker", "Lantau Peak",
    "Tai Mo Shan", "Sharp Peak", "Sunset Peak", "Needle Hill", "Ma On Shan",
    "High West", "Pat Sin Leng", "Jardine's Lookout", "The Twins", "Castle Peak",
    "Pineapple Mountain", "Shing Mun Reservoir", "Aberdeen Peak", "Clear Water Bay Peak",
    "Ap Lei Chau Trail", "Cape D’Aguilar", "Sai Wan Peak", "Hawk's Nest", "Tung Chung Valley"
]

areas = ["Central", "Kowloon", "New Territories", "Lantau", "Hong Kong Island"]
price_levels = [1, 2, 3, 4]

bulk_created_ids = []

for name in location_names:
    payload = {
        "name": name,
        "description": f"A scenic location known as {name}.",
        "maps_url": "https://maps.example.com",
        "price_level": random.choice(price_levels),
        "area": random.choice(areas),
    }

    url = f"{BASE_URL}/locations/"
    try:
        res = requests.post(url, json=payload)
        print(f"Created {name} → Status {res.status_code}")
        data = res.json()
        loc_id = data.get("id")
        if loc_id:
            bulk_created_ids.append(loc_id)
    except Exception as e:
        print(f"Error creating {name}: {e}")

time.sleep(0.5)
print("\nTotal created:", len(bulk_created_ids))
bulk_created_ids[:5]


: 

In [ ]:
url = f"{BASE_URL}/locations/"
res = requests.get(url)

print("Status:", res.status_code)
all_locations = res.json()
print("Total locations returned:", len(all_locations))
print(json.dumps(all_locations[:3], indent=2))  # show first 3


: 

In [ ]:
# Example: filter by area and price_level
url = f"{BASE_URL}/locations/?area=Central&price_level=2"
res = requests.get(url)

print("Status:", res.status_code)
filtered_locations = res.json()
print("Filtered locations count:", len(filtered_locations))
print(json.dumps(filtered_locations[:3], indent=2))


: 

In [ ]:
if bulk_created_ids:
    test_location_id = bulk_created_ids[0]
    print("Using test_location_id from bulk_created_ids:", test_location_id)
elif all_locations:
    test_location_id = all_locations[0]["id"]
    print("No bulk_created_ids, using first from all_locations:", test_location_id)
else:
    test_location_id = None
    print("No locations available to test.")

test_location_id


: 

In [ ]:
if not test_location_id:
    raise RuntimeError("No test_location_id available. Run previous cells first.")

url = f"{BASE_URL}/locations/{test_location_id}"
res = requests.get(url)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


: 

In [ ]:
update_payload = {
    "name": "Updated Test Location",
    "price_level": 3,
}

url = f"{BASE_URL}/locations/{test_location_id}"
res = requests.put(url, json=update_payload)

print("Status:", res.status_code)
print(json.dumps(res.json(), indent=2))


: 

In [ ]:
import os

image_path = "test_image.jpg"  # change if needed

if os.path.exists(image_path):
    url = f"{BASE_URL}/locations/{test_location_id}/images"
    with open(image_path, "rb") as f:
        files = {"file": (os.path.basename(image_path), f, "image/jpeg")}
        res = requests.post(url, files=files)

    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print(f"Image file not found: {image_path}. Skipping image upload test.")


: 

In [ ]:
tags_payload = {
    "tags": ["hiking", "scenic", "sunset"]
}

url = f"{BASE_URL}/locations/{test_location_id}/tags"
res = requests.post(url, json=tags_payload)

print("Status:", res.status_code)
tags_response = res.json()
print(json.dumps(tags_response, indent=2))

tag_ids = [t.get("id") for t in tags_response.get("added_tags", []) if t.get("id") is not None]
tag_ids


: 

In [ ]:
if tag_ids:
    tag_id = tag_ids[0]
    url = f"{BASE_URL}/locations/{test_location_id}/tags/{tag_id}"
    res = requests.delete(url)

    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
else:
    print("No tag_ids available to delete.")


: 

In [ ]:
if not USER_ID:
    print("USER_ID is not set. Skipping /locations/recommended test.")
else:
    url = f"{BASE_URL}/locations/recommended?limit=10"
    res = requests.get(url, headers=HEADERS)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))


: 

In [ ]:
cleanup = False  # set to True if you want to delete the test location

if cleanup and test_location_id:
    url = f"{BASE_URL}/locations/{test_location_id}"
    res = requests.delete(url)
    print("Status:", res.status_code)
    print(json.dumps(res.json(), indent=2))
elif not cleanup:
    print("Cleanup is disabled. Set cleanup = True to delete the test location.")
else:
    print("No test_location_id to delete.")


: 